In [1]:
import pandas as pd
import numpy as np
import random

from tqdm.notebook import tqdm

In [2]:
#!pip install geopy

In [3]:
from geopy.distance import geodesic

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("London postcodes.csv", delimiter=',')

In [6]:
df.shape

(314746, 29)

In [7]:
df.sample()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,GridRef,County,District,Ward,...,Population,Households,Built up area,Built up sub-division,Lower layer super output area,Rural/urban,Region,Altitude,London zone,LSOA Code
72815,EN5 2TF,Yes,51.650664,-0.210642,523890,196168,TQ238961,Greater London,Barnet,Underhill,...,30.0,15.0,Greater London,Barnet,Barnet 004F,Urban major conurbation,London,119,5.0,E01000294


In [8]:
df.columns

Index(['Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing',
       'GridRef', 'County', 'District', 'Ward', 'DistrictCode', 'WardCode',
       'Country', 'CountyCode', 'Constituency', 'Introduced', 'Terminated',
       'Parish', 'NationalPark', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code'],
      dtype='object')

In [9]:
rides = pd.DataFrame(columns=['driver_id', 'client_id',\
                              'start', 'start_latitude', 'start_longtitude', \
                              'finish', 'finish_latitude', 'finish_longtitude', \
                              'distance', 'road_time', 'start_time', 'finish_time', 'cost', \
                              'driver_rate', 'category_driver_feedback', 'text_driver_feedback',\
                             'client_rate', 'category_client_feedback', 'text_client_feedback'])
NUM_RIDES = 5000000

Drivers and clients id's

In [10]:
rides['driver_id'] = np.random.randint(low=0, high=2500, size=NUM_RIDES)
rides['client_id'] = np.random.randint(low=0, high=4500, size=NUM_RIDES)

Start and finish points

In [11]:
rides[['start', 'start_latitude', 'start_longtitude']] = df[['Postcode', 'Latitude', 'Longitude']].sample(n=NUM_RIDES, replace=True).reset_index(drop=True)

In [12]:
rides[['finish', 'finish_latitude', 'finish_longtitude']] = df[['Postcode', 'Latitude', 'Longitude']].sample(n=NUM_RIDES, replace=True).reset_index(drop=True)

Start time

In [13]:
def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

start = pd.to_datetime('2010-01-01')
end = pd.to_datetime('2020-01-01')
rides['start_time'] = random_dates(start, end, NUM_RIDES)

Distance between start and finish points

In [14]:
rides['distance'] = [geodesic((x1, y1), (x2, y2)).km for x1, y1, x2, y2 in tqdm(zip(rides['start_latitude'], \
                                                                                              rides['start_longtitude'], \
                                                                                              rides['finish_latitude'], \
                                                                                              rides['finish_longtitude']), total=NUM_RIDES)]
rides['distance'] = rides['distance'].round(2)

  0%|          | 0/5000000 [00:00<?, ?it/s]

Calculate road time

In [15]:
rides['road_time'] = abs(np.random.normal(size=NUM_RIDES, scale=10)) + rides['distance'] * abs(np.random.normal(size=NUM_RIDES, loc=1, scale=0.25))
rides['road_time'] = rides['road_time'].astype('int')
rides['road_time'] = pd.to_timedelta(rides['road_time'], unit='m')

Calculate finish time

In [16]:
rides['finish_time'] = rides['start_time'] + rides['road_time']

Calculate cost of the ride

In [17]:
def count_cost(start_time, distance):
    cost = 2 + 0.5 * distance
    if (start_time.hour >= 8 and start_time.hour <= 9) or \
        (start_time.hour >= 18 and start_time.hour <= 19):
        cost *= 1.5
    if (start_time.hour >= 22 or start_time.hour <= 6):
        cost *= 1.3
    return cost
    
rides['cost'] = [count_cost(s, d) for s, d in tqdm(zip(rides.start_time, rides.distance), total=NUM_RIDES)]
rides['cost'] = rides['cost'].round(2)

  0%|          | 0/5000000 [00:00<?, ?it/s]

Drivers rates

In [18]:
driver_rate_idx = np.random.randint(low=0, high=NUM_RIDES, size=int(NUM_RIDES*0.3))
driver_rate_distribution_arr = np.random.multinomial(1, [0.2, 0.05, 0.1, 0.25, 0.4], size=int(NUM_RIDES*0.3))
rides['driver_rate'][driver_rate_idx] = np.where(driver_rate_distribution_arr == 1)[1] + 1

In [19]:
driver_feedback_categories_good = ['great service', 'nice car', 'wonderful companion', 'neat and tidy', 'expert navigation', 'recommend']
driver_feedback_categories_bad = ['awful service', 'bad car', 'unpleasant companion', 'dirty', 'non-expert navigation', 'not recommend']

In [20]:
category_driver_good_feedback_idx = np.random.choice(rides[rides.driver_rate > 3].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_driver_feedback"][category_driver_good_feedback_idx] = np.random.choice(driver_feedback_categories_good, size=int(NUM_RIDES*0.3*0.2))

category_driver_bad_feedback_idx = np.random.choice(rides[rides.driver_rate < 4].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_driver_feedback"][category_driver_bad_feedback_idx] = np.random.choice(driver_feedback_categories_bad, size=int(NUM_RIDES*0.3*0.2))

In [21]:
text_good_feedback_driver_length = np.random.randint(low=0, high=7, size=int(NUM_RIDES*0.3*0.2))
text_good_feedback_driver_sample = [random.sample(driver_feedback_categories_good, i) for i in text_good_feedback_driver_length]
rides['text_driver_feedback'][category_driver_good_feedback_idx] = text_good_feedback_driver_sample

text_bad_feedback_driver_length = np.random.randint(low=0, high=7, size=int(NUM_RIDES*0.3*0.2))
text_bad_feedback_driver_sample = [random.sample(driver_feedback_categories_bad, i) for i in text_bad_feedback_driver_length]
rides['text_driver_feedback'][category_driver_bad_feedback_idx] = text_bad_feedback_driver_sample

Clients rates

In [22]:
client_rate_idx = np.random.randint(low=0, high=NUM_RIDES, size=int(NUM_RIDES*0.5))
client_rate_distribution_arr = np.random.multinomial(1, [0.2, 0.05, 0.1, 0.25, 0.4], size=int(NUM_RIDES*0.5))
rides['client_rate'][client_rate_idx] = np.where(client_rate_distribution_arr == 1)[1] + 1

In [23]:
client_feedback_categories_good = ['polite', 'pleasant', 'quiet', 'neat and tidy', 'recommend']
client_feedback_categories_bad = ['unpolite', 'unpleasant', 'loud', 'dirty','not recommend']

In [24]:
category_client_good_feedback_idx = np.random.choice(rides[rides.client_rate > 3].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_client_feedback"][category_client_good_feedback_idx] = np.random.choice(client_feedback_categories_good, size=int(NUM_RIDES*0.3*0.2))

category_client_bad_feedback_idx = np.random.choice(rides[rides.client_rate < 4].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_client_feedback"][category_client_bad_feedback_idx] = np.random.choice(client_feedback_categories_bad, size=int(NUM_RIDES*0.3*0.2))

In [25]:
text_good_feedback_client_length = np.random.randint(low=0, high=6, size=int(NUM_RIDES*0.3*0.2))
text_good_feedback_client_sample = [random.sample(client_feedback_categories_good, i) for i in text_good_feedback_client_length]
rides['text_client_feedback'][category_client_good_feedback_idx] = text_good_feedback_client_sample

text_bad_feedback_client_length = np.random.randint(low=0, high=6, size=int(NUM_RIDES*0.3*0.2))
text_bad_feedback_client_sample = [random.sample(client_feedback_categories_good, i) for i in text_bad_feedback_client_length]
rides['text_client_feedback'][category_client_good_feedback_idx] = text_bad_feedback_client_sample

In [26]:
rides.head()

,driver_id,client_id,start,start_latitude,start_longtitude,finish,finish_latitude,finish_longtitude,distance,road_time,start_time,finish_time,cost,driver_rate,category_driver_feedback,text_driver_feedback,client_rate,category_client_feedback,text_client_feedback
0,1074,3799,W11 4NA,51.509660,-0.210998,HA2 8XJ,51.561740,-0.362215,11.99,0 days 00:29:00,2014-07-05 02:54:18,2014-07-05 03:23:18,10.39,NaN,NaN,NaN,5,NaN,NaN
1,1093,1405,N1 3QY,51.539216,-0.092704,N17 9WT,51.593860,-0.071604,6.25,0 days 00:08:00,2016-10-24 21:22:10,2016-10-24 21:30:10,5.12,NaN,NaN,NaN,NaN,NaN,NaN
2,1061,3990,SE12 9ZN,51.439238,0.025621,DA16 1NJ,51.465819,0.121836,7.31,0 days 00:13:00,2017-08-27 09:45:41,2017-08-27 09:58:41,8.48,5,NaN,NaN,NaN,NaN,NaN
3,1191,2980,W1H 5WD,51.514839,-0.158666,W3 9YU,51.500496,-0.276985,8.37,0 days 00:09:00,2015-02-16 04:16:19,2015-02-16 04:25:19,8.04,NaN,NaN,NaN,NaN,NaN,NaN
4,504,157,TW5 0TS,51.479423,-0.363307,E9 5FL,51.532724,-0.060419,21.85,0 days 00:19:00,2015-08-05 20:59:17,2015-08-05 21:18:17,12.92,1,dirty,"[non-expert navigation, dirty, not recommend, ...",NaN,NaN,NaN


In [27]:
rides.to_csv("data/rides.csv")